<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_4_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1021      0 --:--:-- --:--:-- --:--:--  1021
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [47]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,6,diastolic dysfunction
1,8,ARTHRITIS
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [48]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-4.test.csv",header=None)
test.head()

,0,1
0,4,purple sploches around waist
1,6,stage 1 diastolic dysfunction
2,8,arthritis
3,12,shingles
4,16,flu


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [53]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [57]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.451276,4.782968,0.234638,11:13


In [58]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.655484,4.476473,0.263458,11:07


In [59]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.355937,4.442091,0.266938,10:58


In [60]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.309762,4.417062,0.268330,11:02


In [0]:
learn.save_encoder('pubmed_fold4_first')

In [62]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.693251,3.253030,0.443237,01:07


In [63]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.977884,2.888971,0.506039,01:20


In [64]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.407976,2.623207,0.548309,03:26


In [65]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.253559,2.410095,0.589372,03:20
1,1.985878,2.324460,0.607488,03:33


In [66]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.975380,2.217505,0.611111,03:32
1,1.791289,2.158170,0.635266,03:39


In [67]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.716086,2.110783,0.649758,03:25
1,1.621339,2.057241,0.653382,03:30


In [68]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.525768,2.057487,0.650966,03:28
1,1.446810,2.019398,0.654589,03:14


In [69]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.461633,1.995053,0.659420,03:32
1,1.363158,1.971349,0.658213,03:34


In [70]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.368265,1.992180,0.643720,03:32
1,1.358497,1.938606,0.664251,03:20
2,1.300860,1.963360,0.669082,03:31
3,1.174142,1.923046,0.669082,03:16


In [71]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.193296,1.971162,0.670290,03:30
1,1.215820,1.924178,0.672705,03:19
2,1.193295,1.931041,0.682367,03:19
3,1.080179,1.913213,0.687198,03:38


In [72]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.131599,1.934122,0.679952,03:27
1,1.132558,1.956828,0.688406,03:37
2,1.097104,1.974088,0.683575,03:37
3,1.040033,1.950852,0.683575,03:35


In [73]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.996726,1.985700,0.667874,03:36
1,1.087190,1.997794,0.675121,03:21
2,1.092485,2.043175,0.673913,03:34
3,1.044557,2.015443,0.681159,03:24


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.020077,2.002562,0.678744,03:21
1,1.041160,2.088464,0.672705,03:34
2,1.056916,2.042401,0.682367,03:33
3,0.977172,2.044607,0.681159,03:16


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.002392,2.010858,0.675121,03:20
1,0.991358,2.036022,0.676328,03:19
2,1.023584,2.014897,0.681159,03:21
3,0.971821,1.968085,0.682367,03:29


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.979198,2.007488,0.677536,03:22
1,1.049482,2.063875,0.669082,03:14
2,0.969935,2.029922,0.677536,03:15
3,0.922424,2.005548,0.679952,03:30


In [0]:
# save the best model

learn.save_encoder('ask-4-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [79]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[933, 6, 8, 710, 16, 359, 646, 646, 27, 809, 31, 31, 31, 31, 31, 710, 415, 117, 37, 37, 775, 289, 37, 42, 48, 475, 49, 49, 49, 49, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 53, 710, 53, 710, 79, 521, 61, 61, 62, 64, 76, 76, 76, 289, 289, 710, 710, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 182, 289, 80, 80, 788, 268, 450, 710, 92, 92, 92, 94, 864, 847, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 122, 94, 94, 475, 710, 761, 100, 100, 100, 107, 108, 108, 710, 111, 111, 289, 111, 730, 111, 79, 113, 116, 449, 120, 710, 710, 125, 630, 630, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 129, 131, 710, 140, 143, 553, 157, 160, 165, 178, 164, 710, 168, 172, 172, 289, 172, 172, 172, 172, 624, 165, 470, 741, 20, 182, 182, 182, 182, 211, 183, 710, 183, 183, 183, 183, 183, 183, 183, 183, 186, 164, 710, 188, 194, 194, 40, 289, 190, 190, 190, 190, 462, 289, 426, 197, 205, 206, 207, 208, 210, 16

In [81]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

463
0.5346420323325635
